# Tutorial for AFMpy.LAFM

## Imports

In [ ]:
# Standard library imports
import json
import logging

# Third party imports
import matplotlib.pyplot as plt

# AFMpy imports
from AFMpy import Stack, Plotting

## Logging Configuration

Each module in AFMpy contains logging to for debugging purposes via the default python logging library. Logging for the modules should always be configured at the application level. Included in these tutorials are example logging configuration files that can be loaded with the following functions. You may adjust these logging configuration files as you see fit.

In [ ]:
# Load the preconfigured logging settings
with open('logs/LAFM_Tutorial_LoggingConfig.json', 'r') as f:
    LOGGING_CONFIG = json.load(f)

# Set up the logging configuration
logging.config.dictConfig(LOGGING_CONFIG)

## Matplotlib Configuration

Included within the ```Plotting``` module are functions for creating the high quality figures. A default configuration that matches the figures in the publication is activated by running the following function.

In [ ]:
# Configure the rcParams for the plots
Plotting.configure_formatting()

## Load the Stacks

Here we demonstrate how to load a compressed pickle file containing stack information. Recall from the ```SimAFM_Tutorial``` that stack pickles should be distributed alongside a cryptographic signature (```.sig```) and public key (```.pub```). To load the file call ```Stack.load_compressed_pickle``` specifying the ```pickle_filepath``` to the stack and the ```public_key_filepath``` (both ```str``` or ```Path```). This method will

1. Compute the cryptographic hash of the ```.xz``` file
2. Verify the hash against the corresponding ```.sig``` file using the public key.
3. Proceed to load an instance of the ```Stack``` object **only if** the signature is valid.

If the signature verification fails an exception will be raised. Verification can be bypassed by setting the ```verify_pickle``` (```bool```) to ```False```. However, this practice is **not** recommended. Disabling signature checks leaves you vulnerable to malicious files—use it only at your own risk.

In [ ]:
# Set the filepath for the public key to verify the integrity of the stacks.
PUBLIC_KEY_FILEPATH = '../common/keys/Tutorial_Public.pub'

# Load the cytoplasmic and periplasmic stacks
cytoplasmic_stack = Stack.Stack.load_compressed_pickle(pickle_filepath = '../common/stacks/Example_AC-20-4.xz',
                                                       public_key_filepath = PUBLIC_KEY_FILEPATH)

periplasmic_stack = Stack.Stack.load_compressed_pickle(pickle_filepath = '../common/stacks/Example_AP-20-4.xz',
                                                       public_key_filepath = PUBLIC_KEY_FILEPATH)

## Processing the Stacks

Now that the Stacks are loaded we can process them. First let's calculate the mean AFM image.

We call the ```calc_mean_image``` method to calculate the mean AFM image. This function calculates the per-pixel average of the image stack with ```numpy```, sets it to the ```mean_image``` attribute, and returns it. This method implements a form of caching the image. If the ```mean_image``` attribute is already set (i.e. ```mean_image is not None```), the image is not recalculated unless ```force_recalc``` (```bool```) is ```True```.

In [ ]:
# Calculating the Mean image for each stack.
cytoplasmic_mean_image = cytoplasmic_stack.calc_mean_image()

periplasmic_mean_image = periplasmic_stack.calc_mean_image()

Next is calculating the LAFM image.

We call the ```calc_LAFM_image``` method to calculate the LAFM image which requires two parameters. ```target_resolution``` (```tuple[int,int]```) defines the output resolution that the LAFM image should be upscaled to; ```target_resolution``` must be greater than the resolution of the input stack. ```sigma``` (```float```) is the width of the gaussian broadening applied to each detected local maxima. After LAFM calculation, the ```LAFM_image``` attribute is set and returned. As before, this function implements caching such that if the ```LAFM_image``` attribute is already set, it is not recalculated unless ```force_recalc``` (```bool```) is ```True```.

In [ ]:
# Set the target resolution and gaussian width for LAFM processing.
target_resolution = (96,96)
sigma = 2.25

# Computer the LAFM images.
cytoplasmic_lafm_image = cytoplasmic_stack.calc_LAFM_image(target_resolution = target_resolution, sigma = sigma)
periplasmic_lafm_image = periplasmic_stack.calc_LAFM_image(target_resolution = target_resolution, sigma = sigma)

## Plot the Images

Finally, we display the generated mean and LAFM images for both the cytoplasmic and periplasmic stack.

In [ ]:
fig, ax = plt.subplots(2,2, figsize = (8,8))
for axis in ax.ravel():
    axis.set_xticks([])
    axis.set_yticks([])
ax[0,0].imshow(cytoplasmic_mean_image, cmap=Plotting.LAFMcmap)
ax[0,0].set_title('Mean Image', fontsize = 16)
ax[0,0].set_ylabel('Cytoplasmic', fontsize = 16)
ax[0,1].imshow(cytoplasmic_lafm_image, cmap=Plotting.LAFMcmap)
ax[0,1].set_title('LAFM Image')
ax[1,0].imshow(periplasmic_mean_image, cmap=Plotting.LAFMcmap)
ax[1,0].set_ylabel('Periplasmic', fontsize = 16)
ax[1,1].imshow(periplasmic_lafm_image, cmap=Plotting.LAFMcmap)

plt.show()